In [ ]:
import pandas as pd
import numpy as np

# -------------------------------------------------------------------------------------------------------
# Script 1: Diagnóstico inicial
# -------------------------------------------------------------------------------------------------------

# --- Configuración ---
FILE_PATH = r"..\denuncias_2020_2025.csv"

# --- Inicio del Script ---
print("Iniciando el diagnóstico inicial del archivo CSV (lectura completa)...")
print(f"Archivo: {FILE_PATH}")
print("="*80)

# Leer TODO el CSV en un DataFrame
df = pd.read_csv(
    FILE_PATH,
    sep=",",                # separador fijo
    on_bad_lines="warn",
    low_memory=False,
    na_values=["", " ", "  ", "NA", "N/A", "na", "n/a", "NULL", "null", "Sin dato", "SIN DATO"]
)

# Normalizar blancos a NaN
df = df.replace(r"^\s*$", np.nan, regex=True)

# --- 1) Información General y Tipos de Datos ---
print("## 1) Información General y Tipos de Datos")
print(f"* Filas totales: {len(df):,}")
print(f"* Columnas: {df.shape[1]}")
mem_mb = df.memory_usage(deep=True).sum() / 1024**2
print(f"* Uso de memoria aprox.: {mem_mb:.2f} MB")

info_df = pd.DataFrame({
    "Columna": df.columns,
    "Valores No Nulos": df.count().values,
    "Tipo de Dato": [str(t) for t in df.dtypes.values]
})

# --- 2) Resumen de Valores Faltantes ---
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df_sorted = (
    pd.DataFrame({
        "Columna": df.columns,
        "Valores Faltantes": missing_values.values,
        "% Faltante": missing_percentage.values
    })
    .query("`Valores Faltantes` > 0")
    .sort_values(by="% Faltante", ascending=False)
)

# --- 3) Estadísticas Descriptivas (Numéricas) ---
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
desc_num = df[num_cols].describe().T if num_cols else pd.DataFrame()

# --- 4) Estadísticas Descriptivas (Categóricas) ---
obj_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
desc_cat = df[obj_cols].describe().T if obj_cols else pd.DataFrame()

# --- 5) Distribución para Categorías Clave (Top 10) ---
key_cols = ["departamento_hecho", "tipo_hecho", "turno_hecho", "ano_hecho", "mes_hecho"]
distribuciones = {}
for col in key_cols:
    if col in df.columns:
        distribuciones[col] = (df[col].value_counts(normalize=True, dropna=False).head(10) * 100).round(2)

print("\nDiagnóstico listo. Variables útiles en memoria:")
print(" - df                      → DataFrame completo para seguir trabajando")
print(" - info_df                 → tabla con no nulos y tipos")
print(" - missing_df_sorted       → columnas con faltantes (ordenadas por %)")
print(" - desc_num / desc_cat     → describe() numérico y categórico")
print(" - distribuciones[col]     → top-10 % para cada columna en key_cols presente")


Iniciando el diagnóstico inicial del archivo CSV (lectura completa)...
Archivo: C:\Users\diego\OneDrive\9NO CICLO\ANALÍTICA DE DATOS\PC1\Data\denuncias_2020_2025.csv
## 1) Información General y Tipos de Datos
* Filas totales: 7,425,530
* Columnas: 60
* Uso de memoria aprox.: 14474.90 MB

Diagnóstico listo. Variables útiles en memoria:
 - df                      → DataFrame completo para seguir trabajando
 - info_df                 → tabla con no nulos y tipos
 - missing_df_sorted       → columnas con faltantes (ordenadas por %)
 - desc_num / desc_cat     → describe() numérico y categórico
 - distribuciones[col]     → top-10 % para cada columna en key_cols presente


In [25]:
# ===============================
# Extras para mostrar resultados
# ===============================

# 1) Primeras filas de la data
print("\n### Vista preliminar de los datos:")
print(df.head(3))  # muestra las 3 primeras filas

# 2) Top columnas con más faltantes
print("\n### Columnas con más valores faltantes:")
print(missing_df_sorted.head(10))

# 3) Estadísticas numéricas básicas
if not desc_num.empty:
    print("\n### Resumen estadístico de variables numéricas:")
    print(desc_num.head())

# 4) Ejemplo de distribución categórica
ejemplo_col = "tipo_hecho"
if ejemplo_col in distribuciones:
    print(f"\n### Distribución de '{ejemplo_col}' (Top 10 categorías en %):")
    print(distribuciones[ejemplo_col])


### Vista preliminar de los datos:
      fecha_hora_hecho  mes_hecho  dia_hecho departamento_hecho  \
0  2020-03-27 05:10:00          3         27        LA LIBERTAD   
1  2020-03-28 05:10:00          3         28          CAJAMARCA   
2  2020-03-28 05:00:00          3         28        LA LIBERTAD   

  provincia_hecho distrito_hecho               tipo_hecho  id_tipo_hecho  \
0          ASCOPE         PAIJAN  INTERVENCION POLICIALES            602   
1            JAEN           JAEN  INTERVENCION POLICIALES            602   
2        TRUJILLO       TRUJILLO  INTERVENCION POLICIALES            602   

                materia_hecho  id_materia_hecho  ...  \
0  HECHOS DE INTERES POLICIAL                 6  ...   
1  HECHOS DE INTERES POLICIAL                 6  ...   
2  HECHOS DE INTERES POLICIAL                 6  ...   

   fecha_hora_registro_hecho  barrio  comisaria  departamento  provincia  \
0              1585409365000     NaN        NaN           NaN        NaN   
1            

In [26]:
import time

# -------------------------------------------------------------------------------------------------------
# Script 2: Manejo de valores faltantes
# -------------------------------------------------------------------------------------------------------

# --- Configuración ---
COLUMNS_TO_DROP = [
    'tipologias_ia',
    'cuadra_hecho',
    'barrio',
    'comisaria',
    'departamento',
    'provincia',
    'distrito',
    'indice_priorizacion',
    'fecha_inaguracion'
]

print("Iniciando Paso 2: Manejo de valores faltantes (Eliminación de columnas).")
print(f"Columnas a eliminar: {COLUMNS_TO_DROP}")
print("="*80)

start_time = time.time()

# Usar inplace=True para modificar df directamente
df.drop(columns=COLUMNS_TO_DROP, inplace=True, errors='ignore')

end_time = time.time()
print("Proceso completado.")
print(f"Tiempo total de procesamiento: {end_time - start_time:.2f} segundos.")
print(f"El DataFrame ahora tiene {df.shape[1]} columnas y {df.shape[0]:,} filas.")


Iniciando Paso 2: Manejo de valores faltantes (Eliminación de columnas).
Columnas a eliminar: ['tipologias_ia', 'cuadra_hecho', 'barrio', 'comisaria', 'departamento', 'provincia', 'distrito', 'indice_priorizacion', 'fecha_inaguracion']
Proceso completado.
Tiempo total de procesamiento: 1.86 segundos.
El DataFrame ahora tiene 51 columnas y 7,425,530 filas.


In [27]:
import time
import pandas as pd

# -------------------------------------------------------------------------------------------------------
# Script 3: Transformar columnas
# -------------------------------------------------------------------------------------------------------

COLUMNS_TO_DROP = [
    'fecha_hecho',
    'hora_hecho'
]

print("Iniciando Paso 3: Transformación de Columnas (Fechas) sobre df en memoria.")
print(f"Columnas a eliminar: {COLUMNS_TO_DROP}")
print("="*80)

start_time = time.time()

# 1) Convertir 'fecha_hora_hecho' a datetime (coerce → valores inválidos pasan a NaT)
if 'fecha_hora_hecho' in df.columns:
    df['fecha_hora_hecho'] = pd.to_datetime(df['fecha_hora_hecho'], errors='coerce')
else:
    print("Aviso: la columna 'fecha_hora_hecho' no existe en df; no se aplicó la conversión.")

# 2) Eliminar columnas de fecha/hora redundantes
df.drop(columns=COLUMNS_TO_DROP, inplace=True, errors='ignore')

end_time = time.time()
print("Proceso completado.")
print(f"Tiempo total de procesamiento: {end_time - start_time:.2f} segundos.")
print(f"Dimensiones actuales de df: {df.shape[0]:,} filas x {df.shape[1]} columnas.")

Iniciando Paso 3: Transformación de Columnas (Fechas) sobre df en memoria.
Columnas a eliminar: ['fecha_hecho', 'hora_hecho']
Proceso completado.
Tiempo total de procesamiento: 3.37 segundos.
Dimensiones actuales de df: 7,425,530 filas x 49 columnas.


In [28]:
import time
import pandas as pd

# -------------------------------------------------------------------------------------------------------
# Script 4: Optimizar tipos
# -------------------------------------------------------------------------------------------------------

DTYPE_MAPPING = {
    'mes_hecho': 'int8',
    'dia_hecho': 'int8',
    'id_materia_hecho': 'int8',
    'id_dpto_hecho': 'int8',
    'solo_denuncia': 'int8',
    'estado': 'int8',
    'ano_hecho': 'int16',
    'id_tipo_hecho': 'int16',
    'lat': 'float32',
    'lon': 'float32',
    'lat_hecho': 'float32',
    'long_hecho': 'float32',
}

print("Iniciando Paso 4: Optimización de Tipos de Datos (Numéricos) sobre df en memoria.")
print("="*80)

# --- Medir memoria antes ---
mem_before = df.memory_usage(deep=True).sum() / 1024**2
print(f"Uso de memoria ANTES de optimizar: {mem_before:.2f} MB")

start_time = time.time()

# Aplicar conversiones
for col, dtype in DTYPE_MAPPING.items():
    if col in df.columns:
        try:
            df[col] = df[col].astype(dtype)
        except Exception as e:
            print(f"  - Advertencia: No se pudo convertir la columna '{col}'. Error: {e}")
    else:
        print(f"  - Aviso: La columna '{col}' no existe en df; se omite la conversión.")

end_time = time.time()

# --- Medir memoria después ---
mem_after = df.memory_usage(deep=True).sum() / 1024**2
reduction_pct = ((mem_before - mem_after) / mem_before * 100) if mem_before > 0 else 0

# Resumen
print("\nProceso completado.")
print(f"Tiempo total de procesamiento: {end_time - start_time:.2f} segundos.")
print(f"Dimensiones actuales de df: {df.shape[0]:,} filas x {df.shape[1]} columnas.")
print(f"Uso de memoria DESPUÉS de optimizar: {mem_after:.2f} MB")
print(f"Reducción de memoria: {mem_before - mem_after:.2f} MB ({reduction_pct:.2f}%)")


Iniciando Paso 4: Optimización de Tipos de Datos (Numéricos) sobre df en memoria.
Uso de memoria ANTES de optimizar: 11663.10 MB

Proceso completado.
Tiempo total de procesamiento: 0.12 segundos.
Dimensiones actuales de df: 7,425,530 filas x 49 columnas.
Uso de memoria DESPUÉS de optimizar: 11167.39 MB
Reducción de memoria: 495.71 MB (4.25%)


In [29]:
import time
import numpy as np
import pandas as pd

# -------------------------------------------------------------------------------------------------------
# Script 5: Verificar tipos
# -------------------------------------------------------------------------------------------------------

# --- Configuración ---
NUMERIC_COLS = [
    'mes_hecho', 'dia_hecho', 'id_tipo_hecho', 'id_materia_hecho',
    'ano_hecho', 'id_dpto_hecho', 'solo_denuncia', 'estado',
    'lat', 'lon', 'lat_hecho', 'long_hecho'
]
CATEGORICAL_COLS = [
    'departamento_hecho', 'provincia_hecho', 'distrito_hecho',
    'tipo_hecho', 'materia_hecho', 'turno_hecho', 'es_delito_x',
    'macroregpol_hecho', 'regionpol_hecho', 'estado_coord'
]

print("Iniciando Paso 5: Verificación de Tipos de Datos sobre df en memoria.")
print("Esto puede tardar algunos segundos con ~7M filas…")
print("="*80)

start_time = time.time()

# Filtra a solo columnas existentes para evitar warnings innecesarios
num_exist = [c for c in NUMERIC_COLS if c in df.columns]
cat_exist = [c for c in CATEGORICAL_COLS if c in df.columns]

# --- Rangos (min/max) para numéricas ---
if num_exist:
    # .agg(['min','max']) es vectorizado y muy eficiente
    num_summary = df[num_exist].agg(['min', 'max']).T
    num_summary.columns = ['min_real', 'max_real']
else:
    num_summary = pd.DataFrame(columns=['min_real', 'max_real'])

# --- Conteo de valores únicos para categóricas ---
if cat_exist:
    cat_nunique = df[cat_exist].nunique(dropna=True)
else:
    cat_nunique = pd.Series(dtype='Int64')

end_time = time.time()
duration = end_time - start_time

# --- Reporte en consola (mismo estilo que tu script original) ---
print(f"\nProceso completado en {duration:.2f} segundos. Total de filas analizadas: {len(df):,}")
print("\n--- Rangos de Columnas Numéricas ---")
print(f"{'Columna':<25} {'Min Real':<20} {'Max Real':<20}")
print("-" * 70)
for col in num_summary.index:
    mn = num_summary.loc[col, 'min_real']
    mx = num_summary.loc[col, 'max_real']
    print(f"{col:<25} {str(mn):<20} {str(mx):<20}")

print("\n--- Conteo de Valores Únicos (Categóricas) ---")
print(f"{'Columna':<25} {'Valores Únicos':<15}")
print("-" * 45)
for col in cat_nunique.index:
    print(f"{col:<25} {int(cat_nunique[col]):<15}")

print("\n" + "="*80)
print("Análisis completado. Usar estos rangos y cardinalidades para validar dtypes/consistencias.")


Iniciando Paso 5: Verificación de Tipos de Datos sobre df en memoria.
Esto puede tardar algunos segundos con ~7M filas…

Proceso completado en 2.72 segundos. Total de filas analizadas: 7,425,530

--- Rangos de Columnas Numéricas ---
Columna                   Min Real             Max Real            
----------------------------------------------------------------------
mes_hecho                 1.0                  12.0                
dia_hecho                 1.0                  31.0                
id_tipo_hecho             101.0                714.0               
id_materia_hecho          1.0                  7.0                 
ano_hecho                 2020.0               2025.0              
id_dpto_hecho             1.0                  25.0                
solo_denuncia             0.0                  1.0                 
estado                    1.0                  3.0                 
lat                       -18.348545           -0.1196843          
lon             

In [30]:
# --- Extras---
print("\n### Top 5 columnas categóricas con más valores únicos:")
print(cat_nunique.sort_values(ascending=False).head())

ejemplo_col = "tipo_hecho"
if ejemplo_col in df.columns:
    print(f"\n### Distribución de '{ejemplo_col}' (Top 10 categorías):")
    print(df[ejemplo_col].value_counts().head(10))

if num_exist:
    print("\n### Percentiles 0.5% y 99.5% de variables numéricas:")
    print(df[num_exist].quantile([0.005, 0.995]).T)


### Top 5 columnas categóricas con más valores únicos:
distrito_hecho        1732
provincia_hecho        196
tipo_hecho              53
regionpol_hecho         29
departamento_hecho      25
dtype: int64

### Distribución de 'tipo_hecho' (Top 10 categorías):
tipo_hecho
INTERVENCION POLICIALES                                        2256255
PATRIMONIO (DELITO)                                            1857941
LEY DE VIOLENCIA CONTRA LA MUJER Y GRUPOS VULNERABLES          1777205
SEGURIDAD PUBLICA (DELITO)                                      345496
FALTAS                                                          335756
VIDA, EL CUERPO Y LA SALUD (DELITO)                             279227
LIBERTAD (DELITO)                                               227486
LEY 30096 DELITOS INFORMATICOS, MODIFICADA POR LA LEY 30171     139735
ADMINISTRACION PUBLICA (DELITO)                                  90046
FAMILIA (DELITO)                                                 25785
Name: count, dtype: 

In [32]:
import time
import unicodedata
import re
import pandas as pd

# -------------------------------------------------------------------------------------------------------
# Script 6: Renombrar columnas
# -------------------------------------------------------------------------------------------------------

def normalize_column_name(name: str) -> str:
    """
    Normaliza un nombre de columna a formato snake_case.
    Ej: 'Año Hecho' -> 'ano_hecho'
    """
    # Transliterar acentos/ñ
    nfkd = unicodedata.normalize('NFKD', str(name))
    name = ''.join(c for c in nfkd if not unicodedata.combining(c))
    # Minúsculas
    name = name.lower()
    # Reemplazar no alfanuméricos por '_'
    name = re.sub(r'[^a-z0-9]+', '_', name)
    # Quitar '_' inicial/final
    name = name.strip('_')
    return name

print("Iniciando Paso 5: Renombrar y Estandarizar Nombres de Columnas (sobre df en memoria).")
print("="*80)
start_time = time.time()

# Construir mapeo old -> new a partir de las columnas actuales del df
new_columns = {col: normalize_column_name(col) for col in df.columns}

# Mostrar cambios propuestos
print("Se aplicarán los siguientes cambios en los nombres:")
changed = False
for old, new in new_columns.items():
    if old != new:
        print(f"- '{old}' -> '{new}'")
        changed = True
if not changed:
    print("(No se encontraron columnas que necesiten cambios)")

# Aplicar renombrado en el DataFrame en memoria
df.rename(columns=new_columns, inplace=True)

# Aviso si quedaron nombres duplicados tras la normalización
dup_cols = df.columns[df.columns.duplicated()].unique().tolist()
if dup_cols:
    print("\nADVERTENCIA: Se detectaron nombres de columnas duplicados después de normalizar:")
    for c in dup_cols:
        print(f"  - '{c}' aparece {sum(df.columns == c)} veces")
    print("Sugerencia: resolver manualmente estas colisiones antes de continuar.")

end_time = time.time()
print("\nProceso completado.")
print(f"Tiempo total de procesamiento: {end_time - start_time:.2f} segundos.")
print(f"Dimensiones actuales de df: {df.shape[0]:,} filas x {df.shape[1]} columnas.")


Iniciando Paso 5: Renombrar y Estandarizar Nombres de Columnas (sobre df en memoria).
Se aplicarán los siguientes cambios en los nombres:
(No se encontraron columnas que necesiten cambios)

Proceso completado.
Tiempo total de procesamiento: 0.00 segundos.
Dimensiones actuales de df: 7,425,530 filas x 49 columnas.


In [33]:
import time
import pandas as pd

# -------------------------------------------------------------------------------------------------------
# Script 7: Verificar unicidad
# -------------------------------------------------------------------------------------------------------

# --- Configuración ---
COLUMN_TO_CHECK = 'objectid'

print(f"Iniciando Verificación de Unicidad para la columna '{COLUMN_TO_CHECK}' (sobre df en memoria).")
print("="*80)
start_time = time.time()

if COLUMN_TO_CHECK not in df.columns:
    raise KeyError(f"La columna '{COLUMN_TO_CHECK}' no existe en df. Columnas disponibles: {list(df.columns)[:10]}...")

total_rows = len(df)
# Contar únicos a nivel global; dropna=False cuenta NaN como una categoría
unique_count = df[COLUMN_TO_CHECK].nunique(dropna=False)

duration = time.time() - start_time

# --- Reporte ---
print("\n" + "="*80)
print("  Reporte de Verificación de Unicidad")
print("="*80)
print(f"Proceso completado en {duration:.2f} segundos.")
print(f"Columna analizada: '{COLUMN_TO_CHECK}'")
print(f"Total de filas analizadas: {total_rows:,}")
print(f"Valores únicos encontrados: {unique_count:,}")

print("\n--- Conclusión ---")
if total_rows == unique_count:
    print("(OK) ¡Confirmado! La columna es un identificador único. No hay duplicados.")
else:
    calculated_duplicates = total_rows - unique_count
    print(f"(ERROR) ¡Atención! La columna NO es un identificador único.")
    print(f"   Se encontraron {calculated_duplicates:,} filas duplicadas basadas en '{COLUMN_TO_CHECK}'.")

    # (Opcional) muestra una vista rápida de IDs duplicados y sus conteos
    dup_counts = df[COLUMN_TO_CHECK].value_counts(dropna=False)
    top_dups = dup_counts[dup_counts > 1].head(10)
    if not top_dups.empty:
        print("\nEjemplos de IDs duplicados (top 10):")
        for idx, cnt in top_dups.items():
            print(f"  - {idx}: {cnt} veces")


Iniciando Verificación de Unicidad para la columna 'objectid' (sobre df en memoria).

  Reporte de Verificación de Unicidad
Proceso completado en 0.39 segundos.
Columna analizada: 'objectid'
Total de filas analizadas: 7,425,530
Valores únicos encontrados: 7,425,530

--- Conclusión ---
(OK) ¡Confirmado! La columna es un identificador único. No hay duplicados.


In [34]:
import time
import pandas as pd

# -------------------------------------------------------------------------------------------------------
# Script 8: Verificar categorías
# -------------------------------------------------------------------------------------------------------

# --- Configuración ---
CATEGORICAL_COLS = [
    'departamento_hecho', 'provincia_hecho', 'distrito_hecho',
    'tipo_hecho', 'materia_hecho', 'turno_hecho', 'es_delito_x',
    'macroregpol_hecho', 'regionpol_hecho', 'estado_coord'
]

print("Iniciando Verificación de Valores Únicos para Columnas Categóricas (sobre df en memoria).")
print("Columnas objetivo:", CATEGORICAL_COLS)
print("="*80)

start_time = time.time()

# Filtrar a las columnas que realmente existen en df
cat_exist = [c for c in CATEGORICAL_COLS if c in df.columns]
missing = [c for c in CATEGORICAL_COLS if c not in df.columns]
if missing:
    print("Aviso: estas columnas no existen en df y serán omitidas:", missing)

# Construir diccionario de valores únicos
final_uniques_dict = {}
for col in cat_exist:
    # dropna() para ignorar NaN; sorted para salida consistente
    uniques = df[col].dropna().unique()
    try:
        # Si viene como dtype 'category', .unique() ya es eficiente
        uniques_list = sorted([str(x) for x in uniques])
    except Exception:
        # Fallback robusto
        uniques_list = sorted(map(str, pd.Series(uniques).astype(str).tolist()))
    final_uniques_dict[col] = uniques_list

duration = time.time() - start_time

# --- Reporte en consola (legible) ---
print("\n" + "="*80)
print("  Diccionario de Valores Únicos por Columna Categórica")
print("="*80)
print(f"Proceso completado en {duration:.2f} segundos.")
for col, values in final_uniques_dict.items():
    print(f"\n--- Columna: '{col}' ({len(values)} valores únicos) ---")
    if len(values) > 20:
        print(values[:10], "...")
    else:
        print(values)

print("\nNota: El diccionario completo quedó en la variable `final_uniques_dict` para uso posterior.")


Iniciando Verificación de Valores Únicos para Columnas Categóricas (sobre df en memoria).
Columnas objetivo: ['departamento_hecho', 'provincia_hecho', 'distrito_hecho', 'tipo_hecho', 'materia_hecho', 'turno_hecho', 'es_delito_x', 'macroregpol_hecho', 'regionpol_hecho', 'estado_coord']

  Diccionario de Valores Únicos por Columna Categórica
Proceso completado en 4.47 segundos.

--- Columna: 'departamento_hecho' (25 valores únicos) ---
['AMAZONAS', 'ANCASH', 'APURIMAC', 'AREQUIPA', 'AYACUCHO', 'CAJAMARCA', 'CALLAO', 'CUSCO', 'HUANCAVELICA', 'HUANUCO'] ...

--- Columna: 'provincia_hecho' (196 valores únicos) ---
['ABANCAY', 'ACOBAMBA', 'ACOMAYO', 'AIJA', 'ALTO AMAZONAS', 'AMBO', 'ANDAHUAYLAS', 'ANGARAES', 'ANTA', 'ANTABAMBA'] ...

--- Columna: 'distrito_hecho' (1732 valores únicos) ---
['ABANCAY', 'ABELARDO PARDO LEZAMETA', 'ACARI', 'ACAS', 'ACCHA', 'ACCOMARCA', 'ACHAYA', 'ACHOMA', 'ACO', 'ACOBAMBA'] ...

--- Columna: 'tipo_hecho' (53 valores únicos) ---
['ADMINISTRACION PUBLICA (DELITO)'

In [35]:
import time
import pandas as pd

# -------------------------------------------------------------------------------------------------------
# Script 9: Codificación y Binning
# -------------------------------------------------------------------------------------------------------

print("Iniciando Codificación y Binning.")
print("="*80)
start_time = time.time()

# --- 1) Limpieza para 'estado_coord' ---
estado_coord_replace_map = {
    'SIN COORDENADA XX': 'SIN COORDENADA',
    'SIN COORDENADA YY': 'SIN COORDENADA'
}

# --- 2) Codificación Ordinal para 'turno_hecho' (asume lower-case) ---
turno_hecho_encoding_map = {
    'madrugada': 0,
    'mañana': 1,
    'tarde': 2,
    'noche': 3
}

# --- 3) Binning para la hora del día desde 'fecha_hora_hecho' ---
hour_bins   = [-1, 5, 11, 17, 23]
hour_labels = ['Madrugada', 'Mañana', 'Tarde', 'Noche']

# --- 4) Codificación binaria para 'tiene_coordenada' a partir de 'estado_coord' ---
tiene_coordenada_map = {
    'CON COORDENADA': 1,
    'SIN COORDENADA': 0
}

# --- Diccionario de Datos para documentación (en memoria) ---
data_dictionary = {
    "estado_coord_cleaning": {
        "description": "Se unificaron valores en la columna 'estado_coord'.",
        "mapping": estado_coord_replace_map
    },
    "turno_hecho_encoding": {
        "description": "Codificación ordinal de la columna 'turno_hecho'.",
        "mapping": {"0": "madrugada", "1": "mañana", "2": "tarde", "3": "noche"}
    },
    "periodo_dia_binning": {
        "description": "Nueva columna creada agrupando la hora de 'fecha_hora_hecho'.",
        "bins": {"0-5": "Madrugada", "6-11": "Mañana", "12-17": "Tarde", "18-23": "Noche"}
    },
    "tiene_coordenada_encoding": {
        "description": "Codificación binaria (0/1) creada a partir de 'estado_coord' limpio.",
        "mapping": tiene_coordenada_map
    }
}

# ---------- Aplicación sobre df ----------
# 1) Limpiar 'estado_coord'
if 'estado_coord' in df.columns:
    df['estado_coord'] = df['estado_coord'].replace(estado_coord_replace_map)
else:
    print("Aviso: 'estado_coord' no existe en df; se omite su limpieza.")

# 2) Codificar 'turno_hecho' → 'turno_hecho_cod'
if 'turno_hecho' in df.columns:
    # Asegura lower-case si no lo hiciste antes
    df['turno_hecho_cod'] = (
        df['turno_hecho']
        .astype('string')
        .str.strip()
        .str.lower()
        .map(turno_hecho_encoding_map)
        .astype('Int8')  # permite NaN
    )
else:
    print("Aviso: 'turno_hecho' no existe en df; no se creó 'turno_hecho_cod'.")

# 3) Binning por hora de 'fecha_hora_hecho' → 'periodo_dia'
if 'fecha_hora_hecho' in df.columns:
    # Asegura tipo datetime (Paso 3 ya lo hacía, esto es por si acaso)
    if not pd.api.types.is_datetime64_any_dtype(df['fecha_hora_hecho']):
        df['fecha_hora_hecho'] = pd.to_datetime(df['fecha_hora_hecho'], errors='coerce')
    horas = df['fecha_hora_hecho'].dt.hour  # NaT → NaN
    df['periodo_dia'] = pd.cut(horas, bins=hour_bins, labels=hour_labels, right=True)
else:
    print("Aviso: 'fecha_hora_hecho' no existe en df; no se creó 'periodo_dia'.")

# 4) 'tiene_coordenada' (0/1) a partir de 'estado_coord' limpio
if 'estado_coord' in df.columns:
    df['tiene_coordenada'] = (
        df['estado_coord']
        .astype('string')
        .str.strip()
        .str.upper()
        .map(tiene_coordenada_map)
        .astype('Int8')  # permite NaN cuando no machea
    )
else:
    print("Aviso: 'estado_coord' no existe en df; no se creó 'tiene_coordenada'.")

dur = time.time() - start_time
print("\nProceso completado.")
print(f"Tiempo total de procesamiento: {dur:.2f} segundos.")
print(f"Dimensiones actuales de df: {df.shape[0]:,} filas x {df.shape[1]} columnas.")
print("Variables nuevas/afectadas: 'estado_coord' (limpia), 'turno_hecho_cod', 'periodo_dia', 'tiene_coordenada'.")
print("Diccionario de datos disponible en la variable `data_dictionary`.")


Iniciando Codificación y Binning.

Proceso completado.
Tiempo total de procesamiento: 4.14 segundos.
Dimensiones actuales de df: 7,425,530 filas x 52 columnas.
Variables nuevas/afectadas: 'estado_coord' (limpia), 'turno_hecho_cod', 'periodo_dia', 'tiene_coordenada'.
Diccionario de datos disponible en la variable `data_dictionary`.


In [ ]:
import time

# -------------------------------------------------------------------------------------------------------
# Script 10: Filtrar atípicos
# -------------------------------------------------------------------------------------------------------

# --- Configuración ---
LAT_MIN, LAT_MAX = -18.4, 0
LON_MIN, LON_MAX = -81.4, -68.6

print("Iniciando Paso 10: Filtrado de Valores Atípicos (Geográficos) sobre df en memoria.")
print(f"Límites de Latitud: {LAT_MIN} a {LAT_MAX}")
print(f"Límites de Longitud: {LON_MIN} a {LON_MAX}")
print("="*80)

start_time = time.time()
rows_before = len(df)

# --- Filtrado ---
if 'lat' in df.columns and 'lon' in df.columns:
    df = df[df['lat'].between(LAT_MIN, LAT_MAX) & df['lon'].between(LON_MIN, LON_MAX)]
else:
    raise KeyError("No se encontraron las columnas 'lat' y 'lon' en df.")

rows_after = len(df)
outlier_rows = rows_before - rows_after

duration = time.time() - start_time

print("\n" + "="*80)
print("Proceso de filtrado completado (en memoria).")
print(f"Total de filas procesadas: {rows_before:,}")
print(f"Filas eliminadas como outliers: {outlier_rows:,}")
print(f"Filas finales en df: {rows_after:,}")
print(f"Tiempo total de procesamiento: {duration:.2f} segundos.")

Iniciando Paso 10: Filtrado de Valores Atípicos (Geográficos) sobre df en memoria.
Límites de Latitud: -18.4 a 0
Límites de Longitud: -81.4 a -68.6

Proceso de filtrado completado (en memoria).
Total de filas procesadas: 7,425,530
Filas eliminadas como outliers: 0
Filas finales en df: 7,425,530
Tiempo total de procesamiento: 2.94 segundos.


In [37]:
# --- Extras ---
if outlier_rows > 0:
    print("\n### Ejemplo de coordenadas fuera de rango (primeros 5):")
    # Filas descartadas (outliers)
    mask_outliers = ~(
        df['lat'].between(LAT_MIN, LAT_MAX) & df['lon'].between(LON_MIN, LON_MAX)
    )
    print(df.loc[mask_outliers, ['lat','lon']].head())
else:
    print("\nNo se encontraron outliers geográficos.")

# Mostrar rangos finales
print("\n### Rango de coordenadas tras el filtrado:")
print(f"Latitud: {df['lat'].min()} a {df['lat'].max()}")
print(f"Longitud: {df['lon'].min()} a {df['lon'].max()}")


No se encontraron outliers geográficos.

### Rango de coordenadas tras el filtrado:
Latitud: -18.34854507446289 a -0.11968430131673813
Longitud: -81.3244857788086 a -68.8183364868164


In [38]:
# ==========================================
# Paso 10.1: Depuración final de columnas
# ==========================================
import time

# Si quieres conservar coordenadas/ID por pasos posteriores, activa estas banderas:
KEEP_GEO = False        # True para conservar 'lat' y 'lon'
KEEP_OBJECTID = False   # True para conservar 'objectid'

# Lista original
cols_to_drop_base = [
    'id_tipo_hecho','id_materia_hecho','id_subtipo_hecho','id_modalidad_hecho',
    'id_dpto_hecho','id_prov_hecho','id_dist_hecho','lat','lon',
    'ubigeo_cia_registro','ubigeo_cia_hecho','id_dgc','id_comisaria_registro',
    'cod_uni_hecho','cod_cpnp_hecho','cod_macroregpol_hecho','cod_regpol_hecho',
    'cod_divpol_divopus_hecho','objectid'
]

# Ajuste según banderas
cols_to_drop = cols_to_drop_base.copy()
if KEEP_GEO:
    for c in ['lat','lon']:
        if c in cols_to_drop:
            cols_to_drop.remove(c)
if KEEP_OBJECTID and 'objectid' in cols_to_drop:
    cols_to_drop.remove('objectid')

print("Paso 10.1: Depuración final de columnas (en memoria).")
print("Columnas objetivo (tras aplicar banderas):")
print(cols_to_drop)
print("="*80)

t0 = time.time()

# Eliminar solo las existentes
existentes = [c for c in cols_to_drop if c in df.columns]
faltantes  = [c for c in cols_to_drop if c not in df.columns]

df.drop(columns=existentes, inplace=True, errors='ignore')

print(f"Eliminadas: {len(existentes)} columnas")
if faltantes:
    print(f"(Aviso) No se encontraron y se omitieron: {faltantes}")
print(f"df actual: {df.shape[0]:,} filas x {df.shape[1]} columnas")
print(f"Tiempo: {time.time() - t0:.2f}s")


Paso 10.1: Depuración final de columnas (en memoria).
Columnas objetivo (tras aplicar banderas):
['id_tipo_hecho', 'id_materia_hecho', 'id_subtipo_hecho', 'id_modalidad_hecho', 'id_dpto_hecho', 'id_prov_hecho', 'id_dist_hecho', 'lat', 'lon', 'ubigeo_cia_registro', 'ubigeo_cia_hecho', 'id_dgc', 'id_comisaria_registro', 'cod_uni_hecho', 'cod_cpnp_hecho', 'cod_macroregpol_hecho', 'cod_regpol_hecho', 'cod_divpol_divopus_hecho', 'objectid']
Eliminadas: 19 columnas
df actual: 7,425,530 filas x 33 columnas
Tiempo: 1.33s


In [39]:
# ==========================================
# Script 10.2: Ajuste de fecha_hora_hecho y limpieza de columnas derivadas
# ==========================================
import time
import pandas as pd

print("Iniciando Paso 10.2: Conversión de 'fecha_hora_hecho' a datetime y eliminación de columnas derivadas.")
print("="*80)
start_time = time.time()

# 1) Convertir 'fecha_hora_hecho' a datetime (coerce para errores → NaT)
if 'fecha_hora_hecho' in df.columns:
    df['fecha_hora_hecho'] = pd.to_datetime(df['fecha_hora_hecho'], errors='coerce')
else:
    print("Aviso: 'fecha_hora_hecho' no existe en df.")

# 2) Eliminar columnas redundantes de fecha
cols_to_remove = ['mes_hecho','dia_hecho','ano_hecho']
existentes = [c for c in cols_to_remove if c in df.columns]

if existentes:
    df.drop(columns=existentes, inplace=True, errors='ignore')
    print(f"Columnas eliminadas: {existentes}")
else:
    print("No se encontraron columnas ['mes_hecho','dia_hecho','ano_hecho'] para eliminar.")

end_time = time.time()

print("\nProceso completado.")
print("dtype de 'fecha_hora_hecho' ->", df['fecha_hora_hecho'].dtype)
print(f"Columnas totales en df -> {df.shape[1]}")
print(f"Tiempo de procesamiento: {end_time - start_time:.2f} segundos.")


Iniciando Paso 10.2: Conversión de 'fecha_hora_hecho' a datetime y eliminación de columnas derivadas.
Columnas eliminadas: ['mes_hecho', 'dia_hecho', 'ano_hecho']

Proceso completado.
dtype de 'fecha_hora_hecho' -> datetime64[ns]
Columnas totales en df -> 30
Tiempo de procesamiento: 1.69 segundos.


In [40]:
# ==========================================
# Script 10.3: Eliminación de columnas administrativas irrelevantes
# ==========================================
import time

print("Iniciando Paso 10.3: Eliminación de columnas administrativas irrelevantes.")
print("="*80)
start_time = time.time()

cols_to_drop = [
    "fecha_hora_registro_hecho", "turno_hecho_cod",
    "estado", "fuente", "observacion",
    "estado_coord", "tiene_coordenada"
]

# Eliminar solo las que existan en df
existentes = [c for c in cols_to_drop if c in df.columns]
faltantes = [c for c in cols_to_drop if c not in df.columns]

if existentes:
    df.drop(columns=existentes, inplace=True, errors='ignore')
    print(f"Columnas eliminadas: {existentes}")
else:
    print("No se encontraron columnas administrativas para eliminar.")

if faltantes:
    print(f"(Aviso) Estas columnas no estaban en df y se omitieron: {faltantes}")

end_time = time.time()

print("\nProceso completado.")
print(f"Columnas totales en df -> {df.shape[1]}")
print(f"Tiempo de procesamiento: {end_time - start_time:.2f} segundos.")

Iniciando Paso 10.3: Eliminación de columnas administrativas irrelevantes.
Columnas eliminadas: ['fecha_hora_registro_hecho', 'turno_hecho_cod', 'estado', 'fuente', 'observacion', 'estado_coord', 'tiene_coordenada']

Proceso completado.
Columnas totales en df -> 23
Tiempo de procesamiento: 1.07 segundos.


In [41]:
# ==========================================
# Script 11: Guardar dataset completo en CSV
# ==========================================
import time
import os
import pandas as pd

print("Iniciando Paso 11: Guardado en un único archivo (denuncias_final.csv).")
print("="*80)
start_time = time.time()

# Validar columna de provincias
if "provincia_hecho" not in df.columns:
    raise KeyError("Error: La columna 'provincia_hecho' no se encuentra en df.")

# --- Guardar resultados ---
OUTPUT_DIR = r"C:\Users\diego\OneDrive\9NO CICLO\ANALÍTICA DE DATOS\PC1\ENTREGABLES"
os.makedirs(OUTPUT_DIR, exist_ok=True)

OUTPUT_FILE = os.path.join(OUTPUT_DIR, "denuncias_final.csv")

df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")

end_time = time.time()

print("\nProceso completado.")
print(f"Archivo guardado: {OUTPUT_FILE} ({df.shape[0]:,} filas y {df.shape[1]:,} columnas)")
print(f"Tiempo total de procesamiento: {end_time - start_time:.2f} segundos.")


Iniciando Paso 11: Guardado en un único archivo (denuncias_final.csv).

Proceso completado.
Archivo guardado: C:\Users\diego\OneDrive\9NO CICLO\ANALÍTICA DE DATOS\PC1\ENTREGABLES\denuncias_final.csv (7,425,530 filas y 23 columnas)
Tiempo total de procesamiento: 57.20 segundos.
